# Making data useable

### Cleaning a data set

We have a few goals for the data set from Thinkful. It regards article open-access prices.

1. Determine the five most common journals and the total articles for each. 
2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

#### Helpful Regular Expression tricks we will need

remove all trailing spaces - .str.strip()

remove all trailing special characters - \n

change to all one case

In [4]:
file_name = '../../_Datasets/WELLCOME_APCspend2013_forThinkful.csv'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
apc = pd.read_csv(file_name, encoding='ISO-8859-1', header=0, names=['PMID', 'Publisher', 'Journal', 'Article', 'Cost'])
apc2 = apc.copy()
apc2.head(5)

,PMID,Publisher,Journal,Article,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [6]:
apc2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID         1928 non-null object
Publisher    2127 non-null object
Journal      2126 non-null object
Article      2127 non-null object
Cost         2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [7]:
apc2.drop(columns='PMID', inplace=True) # Unique ID, not necessary
apc2['Cost'] = apc2['Cost'].str.lstrip('£') # Cost is full of unnecessary currency symbols
apc2['Cost'] = apc2['Cost'].str.rstrip('$')
apc2['Cost'] = apc2.Cost.apply(float) # The currency symbols had the 'Cost' column as a string, we want a float
apc2.dropna(subset=['Journal'], inplace=True) # Journal had only one NaN value, dropping

"""Tons of dirty string pieces within the 'Journal' column.
PLOS articles/journals were in varying formats. Let's standardize that.
Some versions of the word American were input as Americal.
The use of the ampersand and the word 'and' are not consistent. Let's make that so.
Many common words were input without the first letter. Fixing those issues."""
apc2['Journal'] = apc2['Journal'].str.upper()
apc2['Journal'] = apc2['Journal'].str.replace(r'plos', 'PLoS', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'plosone', 'PLoS One', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'plos one', 'PLoS One', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'plos 1', 'PLoS One', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'americal', 'AMERICAN', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'&', 'and', case=False)
apc2['Journal'] = apc2['Journal'].str.strip()
apc2['Journal'] = apc2['Journal'].str.rstrip('.')
apc2['Journal'] = apc2['Journal'].str.lstrip('THE ')
apc2['Journal'] = apc2['Journal'].str.lstrip('HE ')
apc2['Journal'] = apc2['Journal'].str.replace(r'^ISTORY', 'HISTORY', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'^VIDENCE', 'EVIDENCE', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'^UROPEAN', 'EUROPEAN', case=False)
apc2['Journal'] = apc2['Journal'].str.replace(r'^XPER', 'EXPER', case=False)

In [8]:
# With the strings cleaned up, lets see total article counts - top 5
apc2['Journal'].value_counts()[:5]

PLoS One                           206
JOURNAL OF BIOLOGICAL CHEMISTRY     60
NEUROIMAGE                          29
NUCLEIC ACIDS RESEARCH              26
PLoS PATHOGENS                      24
Name: Journal, dtype: int64

In [10]:
# We also want to know some summary statistics - let's calculate mean, median, and standard deviation
apc2.groupby('Journal').mean()[:15]

,Cost
Journal,
ACADEMY OF NUTRITION AND DIETETICS,2379.540000
ACS CHEMICAL BIOLOGY,1418.186000
ACS CHEMICAL NEUROSCIENCE,1186.800000
ACS NANO,668.140000
"ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY",771.420000
ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL CRYSTALLOGRAPHY,773.740000
ACTA CRYSTALLOGRAPHICA SECTION F: STRUCTURAL BIOLOGY AND CRYSTALLIZATION COMMUNICATIONS,796.635000
"ACTA CRYSTALLOGRAPHICA, SECTION D",757.180000
ACTA CRYSTALLOGRAPHY D,774.190000


In [11]:
apc2.groupby('Journal').median()[:15]

,Cost
Journal,
ACADEMY OF NUTRITION AND DIETETICS,2379.540
ACS CHEMICAL BIOLOGY,1294.590
ACS CHEMICAL NEUROSCIENCE,1186.800
ACS NANO,668.140
"ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY",771.420
ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL CRYSTALLOGRAPHY,773.740
ACTA CRYSTALLOGRAPHICA SECTION F: STRUCTURAL BIOLOGY AND CRYSTALLIZATION COMMUNICATIONS,796.635
"ACTA CRYSTALLOGRAPHICA, SECTION D",757.180
ACTA CRYSTALLOGRAPHY D,774.190


In [13]:
apc2.groupby('Journal').std()[-15:]

,Cost
Journal,
VASCULAR PHARMACOLOGY,NaN
VET. JOURNAL,NaN
VETERINARY MICROBIOLOGY,NaN
VETERINARY PARASITOLOGY,705346.672200
VETERINARY RECORD,NaN
VETERINARY RESEARCH,NaN
VIROLOGY,697.850753
VIROLOGY JOURNAL,NaN
VIRUS RESEARCH,NaN


In [9]:
# Oh, some of the standard deviations are not churning values - they must be singular entries
# Confirming with an example below
apc2[apc2.Journal == 'VODEVO']

,Publisher,Journal,Article,Cost
201,BioMed Central,VODEVO,Early embryonic determination of the sexual di...,1564.5
